In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [21]:
df = pd.read_csv('India_complete.csv', low_memory= False) 
print(df.head())

# cant have Nan values in target variable
df = df.dropna(subset=['O3'])


              datetime StationId                     StationName       City  \
0  2017-11-24 17:00:00     AP001  Secretariat, Amaravati - APPCB  Amaravati   
1  2017-11-24 18:00:00     AP001  Secretariat, Amaravati - APPCB  Amaravati   
2  2017-11-24 19:00:00     AP001  Secretariat, Amaravati - APPCB  Amaravati   
3  2017-11-24 20:00:00     AP001  Secretariat, Amaravati - APPCB  Amaravati   
4  2017-11-24 21:00:00     AP001  Secretariat, Amaravati - APPCB  Amaravati   

            State  Status  PM2.5    PM10    NO    NO2    NOx    NH3   CO  \
0  Andhra Pradesh  Active  60.50   98.00  2.35  30.80  18.25   8.50  0.1   
1  Andhra Pradesh  Active  65.50  111.25  2.70  24.20  15.07   9.77  0.1   
2  Andhra Pradesh  Active  80.00  132.00  2.10  25.18  15.15  12.02  0.1   
3  Andhra Pradesh  Active  81.50  133.25  1.95  16.25  10.23  11.58  0.1   
4  Andhra Pradesh  Active  75.25  116.00  1.43  17.48  10.43  12.03  0.1   

     SO2      O3  AQI AQI_Bucket  
0  11.85  126.40  NaN        NaN 

In [ ]:
X = df.drop(['AQI_Bucket', 'datetime', 'Status', 'StationId', 'StationName', 'City', 'State', 'O3'] , axis=1)
print(X.head())
y = df['O3']
print(y.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


   PM2.5    PM10    NO    NO2    NOx    NH3   CO    SO2  AQI
0  60.50   98.00  2.35  30.80  18.25   8.50  0.1  11.85  NaN
1  65.50  111.25  2.70  24.20  15.07   9.77  0.1  13.17  NaN
2  80.00  132.00  2.10  25.18  15.15  12.02  0.1  12.08  NaN
3  81.50  133.25  1.95  16.25  10.23  11.58  0.1  10.47  NaN
4  75.25  116.00  1.43  17.48  10.43  12.03  0.1   9.12  NaN
0    126.40
1    117.12
2     98.98
3    112.20
4    106.35
Name: O3, dtype: float64


Fit regressor:

In [23]:
regr = RandomForestRegressor(n_estimators=100, random_state=42)

regr.fit(X_train, y_train)


MemoryError: could not allocate 134217728 bytes

In [ ]:
oob_score = regr.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regr.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')